# Linear Regression 

Throughout the notebook we will use the diabetes dataset to understand the maths behind Linear Regression!

In [1]:
from sklearn import datasets
import pandas as pd
import statsmodels.api as sm
import numpy as np

diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

X = pd.DataFrame(diabetes_X)
X.columns = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

y = pd.DataFrame(diabetes_y)
y.columns = ['disease']

## Simple Linear Regression

Simple Linear Regression Equation and Residual Sum of Squares(RSS) is given by:

$$
y = \beta_0 + \beta_1x + \epsilon \\
\hat{y} = \hat{\beta_0} + \hat{\beta_1}x \\
RSS = (y_1 - \hat{\beta_0} - \hat{\beta_1}x_1)^2+(y_2 - \hat{\beta_0} - \hat{\beta_1}x_2)^2 + ... + (y_n - \hat{\beta_0} - \hat{\beta_1}x_n)^2 \\
RSS = \sum_{i=0}^n(y_i - \hat{\beta_0} - \hat{\beta_1}x_i)^2 \\
$$

### Computing $\hat{\beta_0}$ and $\hat{\beta_1}$

To solve for $\beta_0$ and $\beta_1$, the normal Equations for the above are given by:

$$
\frac{\delta{RSS}}{\delta{\hat{\beta_0}}} = -2\sum (y_i - \hat{\beta_0}+\hat{\beta_1}x_i) = 0 \\
\frac{\delta{RSS}}{\delta{\hat{\beta_1}}} = -2x_i\sum (y_i - \hat{\beta_0}+\hat{\beta_1}x_i) = 0 \\
$$

Solving the first normal equation for $\beta_0$:

$$
\frac{\delta{RSS}}{\delta{\hat{\beta_0}}} = -2\sum (y_i - \hat{\beta_0}+\hat{\beta_1}x_i) = 0 \\
\sum{y_i} - n\hat{\beta_0}-\hat{\beta_1}\sum{x_i} = 0 \\
\underline{\hat{\beta_0} = \bar{y} - \hat{\beta_1}\bar{x}} \;\;\;\;\;
where \;
\bar{x} = \frac{\sum{x_i}}{n} \;\;\;
\bar{y} = \frac{\sum{y_i}}{n}
$$

Solving the second normal equation for $\beta_1$:

$$
\frac{\delta{RSS}}{\delta{\hat{\beta_1}}} = \sum{x_i}(y_i - \hat{\beta_0}+\hat{\beta_1}x_i) = 0 \\
\sum{x_i}(y_i - \bar{y} + \hat{\beta_1}\bar{x} - \hat{\beta_1}x_i) = 0 \;\;\;\;
using \; \hat{\beta_0} = \bar{y} - \hat{\beta_1}\bar{x} \\
\sum{x_i}(y_i-\bar{y}) = \hat{\beta_1}\sum{(x_i-\bar{x})x_i} \\
\hat{\beta_1} = \frac{\sum{(y_i-\bar{y})x_i}}{\sum{(x_i-\bar{x})x_i}} \\
\underline{\hat{\beta_1}= \frac{\sum{(y_i-\bar{y})(x_i-\bar{x}})}{\sum{(x_i-\bar{x})^2}}} \;\;\;\;\;
where \;
\sum{(y_i-\bar{y})}\bar{x} = 0 \;\;
\sum{(x_i-\bar{x})}\bar{x} = 0
$$

In [2]:
# For this example we will chose the Age feature solely 

X_age = X.age

X_age = sm.add_constant(X_age, prepend=False)
model = sm.OLS(y, X_age)
results = model.fit()
rss = results.ssr
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                disease   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     16.10
Date:                Fri, 03 Jul 2020   Prob (F-statistic):           7.06e-05
Time:                        21:04:51   Log-Likelihood:                -2539.2
No. Observations:                 442   AIC:                             5082.
Df Residuals:                     440   BIC:                             5091.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
age          304.1831     75.806      4.013      0.0

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [3]:
# Lets verify if based on our formulas of coeff and intercept we reach the same values

y_mean = [np.mean(y.disease)] * 442

y_diff = y.disease - y_mean
x_mean = 0 # Data is 0 mean
coeff = np.sum((y.disease - y_mean) * (X_age.age - x_mean)) / np.sum(X_age.age**2)
print(f"Calculated coeff is {coeff}")

intercept = y_mean[0] - coeff * x_mean
print(f"Calculated intercept is {intercept}")

print()
print("It can be seen that the values calculated using the formulas are same as the values calculated by the statsmodel. \
        Our maths holds up!")

Calculated coeff is 304.1830745282948
Calculated intercept is 152.13348416289594

It can be seen that the values calculated using the formulas are same as the values calculated by the statsmodel.         Our maths holds up!


### Standard Error of Coefficients

Since we don't have the true mean of the $y$ but only sample mean based on $n$ observations, we don't know the true $\beta_0$ and $\beta_1$, we only have $\hat{\beta_0}$ and $\hat{\beta_1}$. Let's see how far are we from true $\beta_0$ and $\beta_1$ by using standard error, which is given by:
$$
SE(\hat{\beta_0})^2 = \sigma^2[\frac{1}{n} + \frac{\bar{x}^2}{\sum_{i=1}^n(x_i-\bar{x})^2}] \\
SE(\hat{\beta_1})^2 = \frac{\sigma^2}{\sum_{i=1}^n(x_i-\bar{x})^2} \;\;\;
$$
where $\sigma^2$ is variance of $\epsilon$ which is the uncontrolled error in $y = \beta_0 + \beta_1x + \epsilon$, which again is something we don't know.

The approximation of $\sigma$ derived from maximum likelihood for a uncorrelated, zero mean, normally distributed $\epsilon$, is given by Residual Standard Error
$$
\sigma = RSE = \sqrt{\frac{RSS}{(n-p-1)}} = \sqrt{\frac{RSS}{(n-2)}} \;\;\;\; p = Number \; of \; features
$$

In [4]:
print(f"Standard error for coeff is {results.bse[0]}")
print(f"Standard error for intercept is {results.bse[1]}")

# Lets try to verify this as well based on the above formula

sigma = np.sqrt(rss/(442-2))

se_coeff = np.sqrt((sigma**2)/np.sum((X_age.age - x_mean)**2))
print(f"Computed standard error for coeff is {se_coeff}")

se_intercept = np.sqrt(sigma**2*(1/442)) # The second term with x_mean can be avoided since x_mean is zero
print(f"Computed standard error for intercept is {se_intercept}")

print()
print("HOORAY! Values are same.") 

Standard error for coeff is 75.8059991270286
Standard error for intercept is 3.605723675064678
Computed standard error for coeff is 75.80599912702861
Computed standard error for intercept is 3.6057236750646777

HOORAY! Values are same.


### Testing Coefficients
In order to know whether a particular feature is important or not, we use the null hypothesis and see if it can be discarded. 

The null hypothesis $H_0$ says that there is no relationship between a feature $X$ and output $Y$, hence $\beta = 0$
If there is relationship between the feature and ouput, then $\beta$ would be sufficiently far from 0. This means that the value of $\beta$ would be atleast some number of standard deviations away from zero which is given by $t-statistic$, resulting in $p-value$ being < 0.05

Based on the above, $t-statistic$ is given by:
$$
t = \frac{\hat{\beta_1} - 0}{SE({\hat{\beta_1}})}
$$

Using this and the degrees of freedom given by (n-p), we can compute the $p-value$ to check if the value of the coefficient is small enough to indicate that there is relationship between feature and output is not due to chance, and null hypothesis can be discarded.

### Confidence Intervals

We know that y is drawn from the Normal Distribution given by:
$$
y \sim N(\beta_0 + \beta_1x, \sigma^2)
$$

As a result $\hat{\beta_0}$ is also normally distributed:
$$
\hat{\beta_0} \sim N(\beta_0, \sigma^2[\frac{1}{n} + \frac{\bar{x}^2}{\sum_{i=1}^n(x_i-\bar{x})^2}]) \\
\hat{\beta_0} \sim N(\beta_0, SE(\hat{\beta_0})^2)
$$

And $\hat{\beta_1}$ is also normally distributed:
$$
\hat{\beta_1} \sim N(\beta_1, \frac{\sigma^2}{\sum_{i=1}^n(x_i-\bar{x})^2}) \\
\hat{\beta_1} \sim N(\beta_1, SE(\hat{\beta_1})^2)
$$

Using the above, the confidence interval for 95% is given by:
$$
\hat{\beta_0} - t_{0.025, n-2}SE(\hat{\beta_0}) \; \leq \beta_0 \leq \; \hat{\beta_0} + t_{0.025, n-2}SE(\hat{\beta_0}) \\
\hat{\beta_1} - t_{0.025, n-2}SE(\hat{\beta_1}) \; \leq \beta_1 \leq \; \hat{\beta_1} + t_{0.025, n-2}SE(\hat{\beta_1})
$$

In [5]:
# Lets also verify the condfidence interval from the above equations

t_95_440 = 1.9654 # Can be retrieved from the t-table online

conf_coeff_low = 304.183074 - (t_95_440 * 75.80599)
conf_coeff_high = 304.183074 + (t_95_440 * 75.80599)
conf_intercept_low = 152.133484 - (t_95_440 * 3.60572)
conf_intercept_high = 152.133484 + (t_95_440 * 3.60572)

print(f"0.025 confidence level for coeff is {conf_coeff_low}")
print(f"0.975 confidence level for coeff is {conf_coeff_high}")
print(f"0.025 confidence level for intercept is {conf_intercept_low}")
print(f"0.975 confidence level for intercept is {conf_intercept_high}")

0.025 confidence level for coeff is 155.193981254
0.975 confidence level for coeff is 453.17216674599996
0.025 confidence level for intercept is 145.046801912
0.975 confidence level for intercept is 159.220166088


In [6]:
# T-statistics from the model
print(f"t-statistic of coeff from the model is {results.tvalues[0]}")
print(f"t-statistic of intercept from the model is {results.tvalues[1]}")

t_coeff = coeff / se_coeff
t_intercept = intercept / se_intercept

print(f"Calculated t-statistic of coeff is {t_coeff}")
print(f"Calculated t-statistic of intercept is {t_intercept}")

print()
print("HOORAY! Values are same.") 

t-statistic of coeff from the model is 4.012651743018033
t-statistic of intercept from the model is 42.192219335877745
Calculated t-statistic of coeff is 4.012651743018033
Calculated t-statistic of intercept is 42.19221933587771

HOORAY! Values are same.


### Model Accuracy

RSE and RSS defined above provide the measure of fit of the model. Higher the values, worse the model fit. However the absolute value of RSE and RSS might not be very intuitive if the scale of Y is not looked at along with it.

$$
RSE = \sqrt{\frac{RSS}{(n-p-1)}} \;\; p = Number \; of \; features
$$



To address it, we use $R^2$, which indicates the amount of variance in the data that is explained by the model. It is given by:

$$
R^2 = \frac{TSS - RSS}{TSS} \\
\underline{R^2 = 1 - \frac{RSS}{TSS}} \;\;\;\; where \; TSS = \sum(y_i - \bar{y})^2
$$

TSS simply indicates the amount of variance in target variable already present. 

In [7]:
print(f"RSS of the model is {results.ssr}")
print(f"TSS of the model is {results.centered_tss}")
print()
print(f"R-sqaure of the model is {results.rsquared}")
print(f"F-score of the model is {results.fvalue}")

RSS of the model is 2528481.7816048963
TSS of the model is 2621009.124434389

R-sqaure of the model is 0.03530218264671636
F-score of the model is 16.101374010745623


Lets verify!

$$
R^2 = 1 - \frac{RSS}{TSS} \;\; = 1 - \frac{2528481.78}{2621009.12} \;\; = 0.36 \\
$$

It seems like R-sqaure values are correct according to our formula.

## Multiple Linear Regresssion

Multiple Linear Regression Equation and Residual Sum of Squares(RSS) for *p* number of predictors/features and *n* number of data points is given by:

$$
\hat{y} = \hat{\beta_0} + \hat{\beta_1}x_1  + \hat{\beta_2}x_2 + ... + \hat{\beta_p}x_p \\
RSS = (y_1 - \hat{\beta_0} - \hat{\beta_1}x_{1,1} - ... - \hat{\beta_p}x_{1,p})^2 + (y_2 - \hat{\beta_0} - \hat{\beta_1}x_{2,1} - ... - \hat{\beta_p}x_{2,p})^2 + ... + (y_n - \hat{\beta_0} - \hat{\beta_1}x_{n,1} - ... - \hat{\beta_p}x_{n,p})^2 \\
RSS = \sum_{i=0}^n(y_i - \hat{\beta_0} - \hat{\beta_1}x_{i,1} - ... - \hat{\beta_1}x_{i,p})^2 \\
$$

Since in multiple linear regression, X is a two dimensional matrix of size (n * (p + 1)) and $\beta$ is a matrix of size ((p + 1) * 1) and Y is a matrix of size n, we will use the matrix notation to write the above equations for MLR

$$
Y = X\beta + \epsilon \\
\hat{Y} = X\hat{\beta} \\
RSS = (Y - X\hat{\beta})^T(Y - X\hat{\beta})
$$

Note that in the above above matrix X and $\beta$ have dimensions of (p+1) instead of p to accomodate for the intercept. Also RSS is of dimension 1 * 1

### Computing $\hat{\beta}$

$$
\frac{\delta{RSS}}{\delta{\hat{\beta}}} = -2X^T(Y - X\hat{\beta}) = 0 \\
-X^TY + X^TX\hat{\beta} = 0 \\
X^TX\hat{\beta} = X^TY \\
\underline{\hat{\beta} = (X^TX)^{-1}X^TY}
$$

### Standard Error of $\hat{\beta}$

Similar to Linear regression before, we find the the standard error for $\hat{\beta}$ which is given by:

$$
SE(\hat{\beta})^2 = \sigma^2(X^TX)^{-1} \\
\underline{SE(\hat{\beta_{j}})^2 = \sigma^2(X^TX)^{-1}_{j,j}} \\
$$

And similarly the approximation of $\sigma$ derived from maximum likelihood for a uncorrelated, zero mean, normally distributed $\epsilon$, is given by Residual Standard Error
$$
\sigma = RSE = \sqrt{\frac{RSS}{(n-p-1)}}
$$

### Model Accuracy

We already know $R^2$ that we can use for testing the model accuracy. Along with that, to accomodate for the number of predictors, we use Adjusted $R^2$. This takes into number of predictors used in the model. This is to help filter out the predictors that might not have significant impact on the model. Their addition might reduce the R-sqaure value a bit but not enough to distinguish it from noise predictors. Its formula is given by:

$$
Adj. R^2 = 1 - \frac{RSS/(n-p-1)}{TSS/(n-1)}
$$

### Testing Model

Similar to how we test the coefficients to see if they reject the null hypothesis and have any relationship with the target, we use the $f-statistic$ to see if the predictors and response have any relationship. Using this, we can discard the null hypothesis which says there is no relationship between the predictors and response. If that is the case then the $f-statistic$ value would be close to 1 and the p-value of $f-statistic$ would be greater than 0.05 and hence not signifcant to reject the null hypotheses. $f-statistic$ is calculated as following:

$$
F = \frac{(TSS - RSS)/p}{RSS/(n-p-1)}
$$

Lets test this out using our Diabetes dataset and using all the 10 predictors

In [8]:
X_OLS = sm.add_constant(X, prepend=False)
model = sm.OLS(y, X_OLS)
results = model.fit()
print(f"RSS of the model is {results.ssr}")
print(f"TSS of the model is {results.centered_tss}")

RSS of the model is 1263983.156255485
TSS of the model is 2621009.124434389


In [9]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                disease   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     46.27
Date:                Fri, 03 Jul 2020   Prob (F-statistic):           3.83e-62
Time:                        21:04:51   Log-Likelihood:                -2386.0
No. Observations:                 442   AIC:                             4794.
Df Residuals:                     431   BIC:                             4839.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
age          -10.0122     59.749     -0.168      0.8

In [10]:
X_OLS_T = X_OLS.transpose()
XX_T = np.dot(X_OLS_T.to_numpy(), X_OLS.to_numpy())
XX_T_inv = np.linalg.inv(XX_T)

In [11]:
# Calculate beta(coefficients)
XX_T_inv_X_T = np.dot(XX_T_inv, X_OLS_T)
beta = np.dot(XX_T_inv_X_T, y)

In [12]:
sigma_sq = results.ssr/(442-10-1)

In [13]:
import math
for i in range(0, 11):
    print(f"Calculated std error for {X_OLS.columns[i]}, "
          f"coefficient is {beta[i][0]:.4f}, "
          f"standard error of coefficient is {math.sqrt(sigma_sq*XX_T_inv[i][i]):.2f}")

Calculated std error for age, coefficient is -10.0122, standard error of coefficient is 59.75
Calculated std error for sex, coefficient is -239.8191, standard error of coefficient is 61.22
Calculated std error for bmi, coefficient is 519.8398, standard error of coefficient is 66.53
Calculated std error for bp, coefficient is 324.3904, standard error of coefficient is 65.42
Calculated std error for s1, coefficient is -792.1842, standard error of coefficient is 416.68
Calculated std error for s2, coefficient is 476.7458, standard error of coefficient is 339.03
Calculated std error for s3, coefficient is 101.0446, standard error of coefficient is 212.53
Calculated std error for s4, coefficient is 177.0642, standard error of coefficient is 161.48
Calculated std error for s5, coefficient is 751.2793, standard error of coefficient is 171.90
Calculated std error for s6, coefficient is 67.6254, standard error of coefficient is 65.98
Calculated std error for const, coefficient is 152.1335, stan

Lets verify other metrics as well!

$$
R^2 = 1 - \frac{RSS}{TSS} \;\; = 1 - \frac{1263983.15}{2621009.12} \;\; = 0.52 \\
Adj. R^2 = 1 - \frac{RSS/(n-p-1)}{TSS/(n-1)} \;\; = 1 - \frac{1263983.15/(442-10-1)}{2621009.12/(432-1)} \;\;
= 1 - \frac{3002.33}{6081.22} \;\; = 1 - 0.493\;\;  = 0.507
$$

It seems like R-sqaure and Adjusted R-sqaure values are correct according to our formula. Lets see F-statistic now

$$
F = \frac{(TSS - RSS)/p}{RSS/(n-p-1)} \;\; = \frac{(2621009.12 - 1263983.15)/10}{1263983.15/(442-10-1)} \;\;
= \frac{135702.59}{2932.67} \;\; = 46.27
$$

WOHOOO!

As can be observed our calculated coefficients and Standard Error for all the coefficients using the formula match exactly! Using that, we can easily verify t-values as well.

## Maximum Likelihood for Simple Linear Regression

Using Maximum Likelihood we can derive the same results that we did in the beginning of the notebook. This is due to the assumptions we have made about random error $\epsilon$
- $\epsilon \sim N(0, \sigma^2)$ and is indepdent across observations and of X
- $\epsilon$ has constant variation

Based on the above, the conditional probability density function of y is:
$$
\Pi_{i=1}^np(y_i|x_i;\beta_0, \beta_1, \sigma^2) 
= \Pi_{i=1}^n\frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{(y_i-(\beta_0+\beta_1x_i))^2}{2\sigma^2}}
$$
The above is called **Likelihood**

Taking the Log of the above:
$$
log\Pi_{i=1}^np(y_i|x_i;\beta_0, \beta_1, \sigma^2) = \sum_{i=1}^nlogp(y_i|x_i;\beta_0, \beta_1, \sigma^2) \\
-\frac{n}{2}log2\pi - nlogs - \frac{1}{2s^2}\sum_{i=1}^n(y_i - (\beta_0 + \beta_1x_i))^2
$$
The above is called **Log Likelihood**

Now we use the derivative of the above to find $\hat{\beta_0}$, $\hat{\beta_1}$, $\hat{\sigma}^2$ that maximizes the log likelihood and consequently the likelihood

$$
\hat{\beta_0} = \bar{y} - \hat{\beta_1}\bar{x} \\
\hat{\beta_1} = \frac{\sum{(y_i-\bar{y})(x_i-\bar{x}})}{\sum{(x_i-\bar{x})^2}} \\
\hat{\sigma}^2 = \frac{1}{n}\sum_{i=1}^n(y_i - ({\hat{\beta_0}} + \hat{\beta_1}x_i))^2
$$

As can be seen from above, the results are same as the ones we got at the beginning.